# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd

from crash_data_analysis import CrashDataAnalysis

In [2]:
%%time
cda = CrashDataAnalysis()
df = cda.crash_dataframe(verbose=True)

Reading file: data/denver_crashes_2023_01_19__14_31.csv
Max timestamp: Wed Jan 18, 6:48 PM (0.97 days ago)
Deadly crashes this year: 3
CPU times: user 11.6 s, sys: 319 ms, total: 11.9 s
Wall time: 12.2 s


In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
# lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
lat_lon = [39.678456222120396, -104.92219658639499]

In [4]:
nearby = cda.crashes_near_point(
    df #[df.reported_date.dt.date >= datetime.date(2022,1,1)]
    , lat_lon[0]
    , lat_lon[1]
    , radius_miles=0.05
)

In [5]:
display_cols = [
    'crash_date'
    , 'top_traffic_accident_offense'
    , 'incident_address'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
]
# nearby[['crash_date_str', 'top_traffic_accident_offense', 'incident_address']].to_clipboard(index=False)
# nearby.to_clipboard()

In [6]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,21,0,0,1,0
2014,27,0,0,0,0
2015,17,0,0,0,0
2016,37,0,0,1,0
2017,27,0,0,0,0
2018,33,0,0,0,0
2019,28,0,1,1,0
2020,18,0,0,0,0
2021,21,0,0,0,0


In [7]:
# df[df.incident_id == 2022639830].squeeze()

In [8]:
nearby[display_cols].tail(5)

,crash_date,top_traffic_accident_offense,incident_address,bicycle_ind,pedestrian_ind,sbi,fatality
2281,2022-09-29,TRAF - ACCIDENT - HIT & RUN,S HOLLY ST / E EVANS AVE,0.0,0.0,False,False
23529,2022-10-13,TRAF - ACCIDENT,2101 S HOLLY ST,0.0,0.0,False,False
17768,2022-10-28,TRAF - ACCIDENT,5600 BLOCK E EVANS AVE,0.0,0.0,False,False
22122,2022-11-02,TRAF - ACCIDENT,2101 S HOLLY ST,0.0,0.0,False,False
24121,2022-11-11,TRAF - ACCIDENT,2100 S HOLLY ST,0.0,0.0,False,False


In [9]:
# len(df[df.reported_date.dt.date >= datetime.date(2022,1,1)])

In [10]:
# sum(df[df.reported_date.dt.date >= datetime.date(2022,1,1)].geo_lon.notnull())

In [11]:
# data_to_map = df[
#     (df.incident_address.str.contains('N SHERIDAN BLVD'))
#     & (df.reported_date.dt.date > datetime.date(2022,12,9))
# ][[
#     'crash_date'
#     , 'top_traffic_accident_offense'
#     , 'incident_address'
#     , 'geo_lon'
#     , 'geo_lat'
# ]]

In [12]:
# create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(
        location=[point.geo_lat, point.geo_lon]
        , radius=2
        , weight=5
        , popup=point.crash_date
    ).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
nearby.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
# this_map.save('html_map_output/simple_dot_plot.html')

# this_map

In [13]:
# data_to_map